[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/taylor-SUNY-Poly/CS295_Public/blob/main/lunar_lander/lunar_lander_colab_port.ipynb)

# Lunar Lander NEAT — Colab Runtime

This notebook sets up a Google Colab runtime for the Lunar Lander neuroevolution lab. It mirrors the command-line workflow that you would follow locally, but leverages Colab's compute resources.

You can use the table below to get a better idea of the cost/speed tradeoffs of the different CPU/GPU options available in Colab.

| Hardware | Config   | Speedup vs Low‑RAM CPU | Cost (CU/hr) |
|:--|:--|--:|--:|
| **CPU** | Low RAM  | **1.00×** | 0.07 |
|         | High RAM | **3.51×** | 0.20 |
| **T4 GPU** | Low RAM  | 1.11× | 1.90 |
|           | High RAM | 4.59× | 1.70 |
| **TPU (v6.e1)** | High RAM | **17.89×** | 3.40 |
| **L4 GPU** | High RAM | 5.86× | 5.37 |
| **A100 GPU** | High RAM | 5.76× | 7.52 |




## 1. Install runtime dependencies

Run the cell below once per Colab session. It installs the system package **swig** (needed by `gymnasium[box2d]`) and the Python libraries referenced by `lunarlander.py` and `report_utils.py`.

In [ ]:
%%bash
set -e
apt-get update -qq
apt-get install -y swig > /dev/null

In [ ]:
!pip install --quiet neat-python "gymnasium[box2d]" imageio matplotlib

## 2. Set the working directory


In [ ]:
%%bash
set -e
REPO_URL="https://github.com/taylor-SUNY-Poly/CS295_Public.git"
TARGET_DIR="/content/CS295_Public"
if [ ! -d "$TARGET_DIR/.git" ]; then
  git clone --depth 1 $REPO_URL $TARGET_DIR
else
  cd $TARGET_DIR
  git pull --ff-only
fi


In [ ]:
from __future__ import annotations
import os
from pathlib import Path

chosen_root = Path('/content/CS295_Public/lunar_lander')

os.chdir(chosen_root)
print(f'Working directory set to: {Path.cwd()}')
print('Available files:')
for item in sorted(Path.cwd().iterdir()):
    print(' -', item.name)


## 3. Edit the Configs
Using the file explorer on the left, navigate to `content/CS295_Public/lunar_lander`.  Double click on a file to open and edit its contents.

## 4. Launch training runs

Assuming you set up Google Colab Pro (you get this for free as a student if you register), you can click on the `Terminal` button at the bottom left of colab.  This will give you access to a proper terminal.  You should use the `cd` and `ls` commands to navigate your way to the `lunare_lander` directory.  From here you can use the CLI commands from the lab instructions to kick off training runs.

## 5. Download your results

The export cell bundles any existing `phase1/`, `phase2/`, and `phase3/` directories into a timestamped ZIP archive, then triggers a download in Colab. IYou should then be able to properly view the generated markdown files, and load genomes to view a visualization of their performance.

In [ ]:
from __future__ import annotations
from datetime import datetime
import zipfile
from pathlib import Path
import os

phase_dirs = [Path(f'phase{i}') for i in range(1, 4)]
existing = [p for p in phase_dirs if p.exists()]

# Add lunarlander.py, report_utils.py, and config files to the list of files to be zipped
files_to_zip = [Path('lunarlander.py'), Path('report_utils.py'), Path('lunarlander_phase1.config'), Path('lunarlander_phase2.config'), Path('lunarlander_phase3.config')]

if not existing and not any(f.exists() for f in files_to_zip):
    print('No phase directories or specified files found yet. Run some training or ensure the files exist!')
else:
    timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
    archive_path = Path(f'lunar_lander_artifacts_{timestamp}.zip')
    with zipfile.ZipFile(archive_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for directory in existing:
            for path in directory.rglob('*'):
                if path.is_file():
                    # Construct the archive name manually
                    archive_name = os.path.join(directory.name, os.path.relpath(path, directory))
                    zf.write(path, arcname=archive_name)
        # Add the specified files to the zip
        for file_path in files_to_zip:
            if file_path.exists():
                 zf.write(file_path, arcname=file_path.name)


    print(f'Created archive: {archive_path}')
    try:
        from google.colab import files
        files.download(str(archive_path))
    except ModuleNotFoundError:
        print('Not running inside Colab; download the ZIP manually.')